In [ ]:
# https://huggingface.co/am-azadi/EXIST2024_Task1_xlmRoberta_large_3_16_merged_unshuffled

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# xlmRoBERTa

In [ ]:
# Load the tokenizer and model
model_name = "am-azadi/EXIST2024_Task1_xlmRoberta_large_3_16_merged_unshuffled"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# use the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the dataset
with open("EXIST2024_dev.json", "r") as file:
    data = json.load(file)

# Function to preprocess and predict, return the soft predictions
def predict_sexism(tweet):
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, max_length=128).to(device)
    outputs = model(**inputs)
    return torch.softmax(outputs.logits, dim=1).tolist()[0][1]

In [ ]:
result = [
    {
        "test_case": "EXIST2024",
        "id": tweet["id_EXIST"],  # Use the id directly from subset_data
        "actual": tweet["labels_task1"].count("YES") / 6,
        "value": {"YES" : predict_sexism(tweet["tweet"]),
                  "NO" : 1 - predict_sexism(tweet["tweet"])}
    }
    for tweet in data.values()
]

In [ ]:
# Save the predictions in a Json file
with open("preds_task1_soft.json", "w") as file:
    json.dump(result, file)